In [1]:
!pip install transformers
!pip install seqeval
!pip install tensorboardx

     |████████████████████████████████| 43 kB 2.1 MB/s eta 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=0bf8e6963af393377716a205ffbee5197d5baa8393d0553066e384a584566114
  Stored in directory: /Users/vprentice/Library/Caches/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval
     |████████████████████████████████| 124 kB 4.7 MB/s eta 0:00:01


In [2]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 21.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 16.3 MB/s eta 0:00:01    |█▋                              | 81 kB 23.2 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 13.7 MB/s eta 0:00:01


  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6488 sha256=024b032da53fa24ebf6d83b105416ac9b7d18565d8cc9b0ddfd5e2d3785af90d
  Stored in directory: /Users/vprentice/Library/Caches/pip/wheels/9f/69/d1/50b39b308a87998eaf5c1d9095e5a5bd2ad98501e2b7936d36
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8784 sha256=b839b2989ed144edae8b2039a1dc866011a65766ecc23e127634b1a40ee2ac19
  Stored in directory: /Users/vprentice/Library/Caches/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built subprocess32 pathtools


In [4]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from happysadsongs.data import *

In [5]:
data = get_training_data()

In [6]:
data.head()

,Unnamed: 0,text,word_label,source
0,0,i am thankful that she continues to feel comfo...,happy,HuggingFace
1,1,i want to do it the right way oh orihime whisp...,happy,HuggingFace
2,2,i was gaining weight getting a lot stronger an...,happy,HuggingFace
3,3,i feel that there is no way to determine if a ...,happy,HuggingFace
4,4,i feel sure that i wouldnt have gained so much...,happy,HuggingFace


In [7]:
def clean2(text):
    # lowercase
    new_text = text.lower()

    # remove twitter handles
    new_text = re.sub(r"@\w+", '', new_text)

    # remove urls
    new_text = re.sub(r'http:\S+', '', new_text)
    new_text = re.sub(r'https:\S+', '', new_text)

    # remove punctuation
#     for punctuation in punctuations:
#         new_text = new_text.replace(punctuation, '')

    # remove numbers
    new_text = ''.join(word for word in new_text if not word.isdigit())

    return new_text

data['clean_text'] = data['text'].apply(clean2)

In [8]:
emotion_dict = {'happy': 0, 'sad': 1, 'angry': 2}
data['label'] = data.word_label.replace(emotion_dict)

In [9]:
data.head()

,Unnamed: 0,text,word_label,source,clean_text,label
0,0,i am thankful that she continues to feel comfo...,happy,HuggingFace,i am thankful that she continues to feel comfo...,0
1,1,i want to do it the right way oh orihime whisp...,happy,HuggingFace,i want to do it the right way oh orihime whisp...,0
2,2,i was gaining weight getting a lot stronger an...,happy,HuggingFace,i was gaining weight getting a lot stronger an...,0
3,3,i feel that there is no way to determine if a ...,happy,HuggingFace,i feel that there is no way to determine if a ...,0
4,4,i feel sure that i wouldnt have gained so much...,happy,HuggingFace,i feel sure that i wouldnt have gained so much...,0


In [10]:
base_data = data[['clean_text','label']]
base_data

,clean_text,label
0,i am thankful that she continues to feel comfo...,0
1,i want to do it the right way oh orihime whisp...,0
2,i was gaining weight getting a lot stronger an...,0
3,i feel that there is no way to determine if a ...,0
4,i feel sure that i wouldnt have gained so much...,0
...,...,...
42480,this hurts to hear because we know it's true. ...,1
42481,yep happening to me all the time at my college...,1
42482,"shit, i lost the original page. please forgive...",1
42483,i want to die,1


In [11]:
train_df, eval_df = train_test_split(base_data, test_size=0.2)

In [16]:
from simpletransformers.classification import ClassificationModel

args={
    'train_batch_size': 8,
    'eval_batch_size': 8,
    'learning_rate': 3e-5, 
    'num_train_epochs': 1, 
    'max_seq_length': 500
    }
model = ClassificationModel('roberta', 'roberta-base', num_labels=4, args=args, use_cuda=False)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.de

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [17]:
model.train_model(train_df)

/Users/vprentice/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:615: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/33988 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/4249 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#     #torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
# model_save_name = 'bert_epoch4.model'
# path = F"/content/gdrive/My Drive/{model_save_name}" 
# torch.save(model.state_dict(), path)